Antes de comenzar, tendremos que para utilizar estas funciones, se necesita la función objetivo $f$, su gradiente $df$, y el hessiano $ddf$. Definiendo la estructura genérica para los métodos de descenso:

* `f`: Función objetivo
* `df`: Gradiente de la función objetivo.
* `ddf`: El hessiano de la función objetivo.
* `x0`: Punto de inicio.
* `a`: Step size.
* `maxIter`: Número máximo de pasos o iteraciones.
* `e`: Tolerancia para el criterio de paro.
* `stopCriterion`: Criterio de paro.

In [20]:
using LinearAlgebra

1. **Descenso Gradiente Naive con Dirección Aleatoria**

In [21]:
function GradNaiveRand(f, df, x0, a, maxIter, e, stopCriterion)
    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]
    errors = []

    for k in 1:maxIter
        # Generar la dirección de descenso al gradiente de forma aleatoria
        d = randn(length(x))

        # Normalizar la dirección de descenso
        d = d/norm(d)

        # Calcular el nuevo punto
        x_new = x + a*d

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x_new))

        # Guardar el error
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualizar el punto actual
        x = x_new    
    end

    # Retornar el punto actual, la secuencia de puntos, la secuencia de valores de la función objetivo, la secuencia de errores, el número de iteraciones
    # Y un indicador que no converge
    return x, x_k, f_k, errors, maxIter, false
end

GradNaiveRand (generic function with 1 method)

2. **Descenso Máximo Naive**

In [22]:
function GradNaiveMax(f, df, x0, a, maxIter, e, stopCriterion)
    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]

    errors = []

    for k in 1:maxIter
        # Calcular la dirección de descenso gradiente
        d = df(x)

        # Asegurarse de que x y d tengan las mismas dimensiones
        if length(x) != length(d)
            throw(DimensionMismatch("x y d deben tener las mismas dimensiones"))
        end

        # Calcular el nuevo punto
        x_new = x + a*d

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x))

        # Guardar el error
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualizar el punto actual
        x = x_new
    end

    # Retornar el punto actual, la secuencia de puntos, la secuencia de valores de la función objetivo, la secuencia de errores, el número de iteraciones
    # Y un indicador que no converge
    return x, x_k, f_k, errors, maxIter, false
end

GradNaiveMax (generic function with 1 method)

3. **Descenso Gradiente de Newton**

In [23]:
function GradNewton(f, df, x0, a, maxIter, e, stopCriterion)
    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]

    errors = []

    # Inicializar la matriz identidad para el Hessiano
    H_I = I(length(x0))

    for k in 1:maxIter
        # Calcular la dirección de descenso
        d = - H_I * df(x)

        # Calcular el nuevo punto
        x_new = x + a*d

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x))
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualización del Hessiano Inverso
        s = x_new - x
        y = df(x_new) - df(x)
        rho = 1/(y'*s)
        H_I = (I -rho*s*y')*H_I*(I-rho*y*s') + rho*s*s'

        # Actualizar el punto actual
        x = x_new

    end

    return x, x_k, f_k, errors, maxIter, false
end

GradNewton (generic function with 1 method)

4. **Descenso Gradiente de Newton (Hessiano Exacto)**

In [24]:
function GradNewtonExact(f, df, ddf, x0, a, maxIter, e, stopCriterion)

    λ = 1e-6  # Regularización

    # Definir el punto inicial
    x = x0

    # x_k es la secuencia de puntos x obtenidos en cada iteración
    x_k = [x0]

    # f_k es la secuencia de valores de la función objetivo en cada iteración
    f_k = [f(x0)]

    errors = []

    for k in 1:maxIter
        # Obtener la dirección de descenso
        g = df(x)
        H = ddf(x) + λ * I

        # Resolver el sistema de ecuaciones
        d = -H\g

        # Calcular el nuevo punto
        x_new = x + a*d

        # Guardar el nuevo punto y el valor de la función objetivo
        push!(x_k, x_new)
        push!(f_k, f(x_new))

        # Calcular el error
        error = norm(df(x))
        push!(errors, error)

        # Si se cumple el criterio de paro, terminar
        if error < e
            return x_new, x_k, f_k, errors, k, true
        end

        # Actualizar el punto actual
        x = x_new
    end

    return x, x_k, f_k, errors, maxIter, false
end

GradNewtonExact (generic function with 1 method)

**Testeando la función $ f : \R^{2} \to \R $ dada por**

$$f(x_1,x_2) = 100(x_2-x_{1}^{2})^{2}+(1-x_{1})^{2}$$

In [25]:
function f(x)
    x1, x2 = x
    return 100*(x2 - x1^2)^2 + (1 - x1)^2
end

f (generic function with 1 method)

**Gradiente de la función**:

El gradiente $\nabla f(x_1,x_2)$

$$ \nabla f(x_1, x_2) = \frac{\partial f}{\partial x_1} = -400x_1(x_2 - x_{1}^{2})-2(1-x_1), \frac{\partial f}{\partial x_2} = 200(x_2 - x_{1}^{2})$$

In [26]:
function df(x)
    x1, x2 = x
    df_dx1 = -400 * x1 * (x2 - x1^2) - 2 * (1 - x1)
    df_dx2 = 200 * (x2 - x1^2)
    return [df_dx1, df_dx2]
end

df (generic function with 1 method)

**Hessiano de la función**:

El hessiano $H(f(x_1,x_2))$

$$ H(f) = 
\begin{bmatrix}
1200 * x_{1}^{2} - 400 * x_2 + 2 & -400 * x_1\\
-400 * x_1 & 200
\end{bmatrix}

In [27]:
function ddf(x)
    x1, x2 = x
    h11 = 1200 * x1^2 - 400 * x2 + 2
    h12 = -400 * x1
    h21 = -400 * x1
    h22 = 200
    return [h11 h12; h21 h22]
end

ddf (generic function with 1 method)

**Punto inicial**: $x_0 = (-1.2, 1)^{T}$


In [28]:
x0 = [-1.2, -1.0] 

2-element Vector{Float64}:
 -1.2
 -1.0

In [29]:
α = 0.001  # Tamaño de paso
maxIter = 10000  # Máximo de iteraciones
ε = 1e-6  # Tolerancia
stop_criterion = :gradiente  # Criterio de paro

:gradiente

### Algoritmo de Descenso Gradiente Naive con Dirección Aleatoria

In [30]:
α = 0.001  # Tamaño de paso
maxIter = 1000000
x_opt, xk, fk, errores, iteraciones, convergencia = GradNaiveRand(f, df, x0, α, maxIter, ε, stop_criterion)
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

Solución óptima: [-1.2660623299215223, -0.3660746384699944]
Número de iteraciones: 1000000
Valor de f en la solución: 392.8265947202847
Convergencia: false


### Algoritmo de Descenso Máximo Naive

In [31]:
α = 0.00001  # Tamaño de paso
maxIter = 1000000
x_opt, xk, fk, errores, iteraciones, convergencia = GradNaiveMax(f, df, x0, α, maxIter, ε, stop_criterion)
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

Solución óptima: [-Inf, -Inf]
Número de iteraciones: 1000000
Valor de f en la solución: Inf
Convergencia: false


### Algoritmo de Descenso Gradiente de Newton

In [32]:
α = 0.0001
maxIter = 1000000
x_opt, xk, fk, errores, iteraciones, convergencia = GradNewton(f, df, x0, α, maxIter, ε, stop_criterion)
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

Solución óptima: [0.9999991158655859, 0.9999982295635587]
Número de iteraciones: 207384
Valor de f en la solución: 7.82163855840224e-13
Convergencia: true


### Algoritmo de Descenso Gradiente de Newton (Hessiano Exacto)

In [33]:
x_opt, xk, fk, errores, iteraciones, convergencia = GradNewtonExact(f, df, ddf, x0, α, maxIter, ε, stop_criterion)
println("Solución óptima: ", x_opt)
println("Número de iteraciones: ", iteraciones)
println("Valor de f en la solución: ", f(x_opt))
println("Convergencia: ", convergencia)

Solución óptima: [0.9999991434244574, 0.9999982848600448]
Número de iteraciones: 209583
Valor de f en la solución: 7.341175124825433e-13
Convergencia: true


**Óptimo**: $x^{*} = (1, 1)^{T}, \ \ \ f(x^{*}) = 0$